# NGS pipeline for Alzheimer's Disease RNA-seq data

For this project I have used only one control and one sample of someone of advanced AD.

In [9]:
import os

Set up some global variables:

In [10]:
# Working directory to the sequencing data (I use an external disk, since I dont have a lot of space left on the pc)
os.environ['WORKDIR'] = '/media/mees/Elements/ngs_data/'
# The directory for the reference genome
os.environ['GENOMEDIR'] = '/media/mees/Elements/ngs_data/genomes/Homo_sapiens/ncbi-genomes-2020-06-22/'

In [7]:
!mkdir -p $WORKDIR
!bash download_data.sh -d $WORKDIR


2020-06-22T17:53:08 prefetch.2.10.7: 1) Downloading 'SRR2422918'...
2020-06-22T17:53:08 prefetch.2.10.7:  Downloading via HTTPS...
^C

2020-06-22T17:53:23 fasterq-dump.2.10.7 int: buffer insufficient while reading uri within cloud module - cannot Get Cloud Location


After downloading the data and convert the .sra files to .fastq files, use FastQC to determine read quality.

In [27]:
!./fastqc.sh -d $WORKDIR 

Started analysis of SRR2422918.fastq
Approx 5% complete for SRR2422918.fastq
Approx 10% complete for SRR2422918.fastq
Approx 15% complete for SRR2422918.fastq
Approx 20% complete for SRR2422918.fastq
Approx 25% complete for SRR2422918.fastq
Approx 30% complete for SRR2422918.fastq
Approx 35% complete for SRR2422918.fastq
Approx 40% complete for SRR2422918.fastq
Approx 45% complete for SRR2422918.fastq
Approx 50% complete for SRR2422918.fastq
Approx 55% complete for SRR2422918.fastq
Approx 60% complete for SRR2422918.fastq
Approx 65% complete for SRR2422918.fastq
Approx 70% complete for SRR2422918.fastq
Approx 75% complete for SRR2422918.fastq
Approx 80% complete for SRR2422918.fastq
Approx 85% complete for SRR2422918.fastq
Approx 90% complete for SRR2422918.fastq
Approx 95% complete for SRR2422918.fastq
Analysis complete for SRR2422918.fastq
Started analysis of SRR2422926.fastq
Approx 5% complete for SRR2422926.fastq
Approx 10% complete for SRR2422926.fastq
Approx 15% complete for SRR2

I like to open the created reports of fastq to see if there are some abnormalities.

In [28]:
!firefox $WORKDIR/*.html

From these reports it seems that the first 10 bp have an over representation of T, therefore we want to trim and remove the first 10 bases. And we are going to use the seqtk package.

In [ ]:
!./seqtk.sh -d $WORKDIR